# CRYPTO-01: Модулярная арифметика

В этом notebook вы закрепите основы модулярной арифметики — фундамента криптографии блокчейнов.

**Темы:**
- Базовые модулярные операции
- Расширенный алгоритм Евклида
- Модулярный обратный элемент
- Малая теорема Ферма
- Быстрое возведение в степень

## 1. Базовые модулярные операции

Оператор `%` в Python выполняет операцию по модулю. Функция `pow(base, exp, mod)` — модулярное возведение в степень.

In [ ]:
# Базовые операции в Z_7
p = 7

# Сложение
print(f"4 + 5 = {(4 + 5) % p} (mod {p})")

# Вычитание (Python корректно обрабатывает отрицательные числа)
print(f"3 - 6 = {(3 - 6) % p} (mod {p})")

# Умножение
print(f"3 * 5 = {(3 * 5) % p} (mod {p})")

# Возведение в степень
print(f"3^4 = {pow(3, 4, p)} (mod {p})")

## 2. Расширенный алгоритм Евклида

Алгоритм находит `gcd(a, b)` и коэффициенты `x, y`, такие что `a*x + b*y = gcd(a, b)`.
Когда `gcd(a, p) = 1`, коэффициент `x` — это модулярный обратный элемент `a^(-1) mod p`.

In [ ]:
def extended_gcd(a: int, b: int) -> tuple[int, int, int]:
    """Расширенный алгоритм Евклида.
    
    Возвращает (gcd, x, y), где a*x + b*y = gcd(a, b)
    """
    if a == 0:
        return b, 0, 1
    
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    
    return gcd, x, y

# Тест: gcd(35, 15) = 5
g, x, y = extended_gcd(35, 15)
print(f"gcd(35, 15) = {g}")
print(f"35 * {x} + 15 * {y} = {35 * x + 15 * y}")
assert 35 * x + 15 * y == g

# Тест: gcd(3, 7) = 1 -> 3^(-1) mod 7
g, x, y = extended_gcd(3, 7)
print(f"\ngcd(3, 7) = {g}")
print(f"3 * {x} + 7 * {y} = {3 * x + 7 * y}")
print(f"3^(-1) mod 7 = {x % 7}")

In [ ]:
def mod_inverse(a: int, p: int) -> int:
    """Вычисляет a^(-1) mod p через расширенный алгоритм Евклида."""
    gcd, x, _ = extended_gcd(a % p, p)
    if gcd != 1:
        raise ValueError(f"Обратный элемент не существует: gcd({a}, {p}) = {gcd}")
    return x % p

# Проверка: совпадает с встроенным pow(a, -1, p)
p = 17
for a in range(1, p):
    our_inv = mod_inverse(a, p)
    python_inv = pow(a, -1, p)
    assert our_inv == python_inv, f"Ошибка для a={a}"
    print(f"{a}^(-1) = {our_inv} (mod {p}), проверка: {a} * {our_inv} = {(a * our_inv) % p}")

print("\nВсе обратные элементы совпадают с pow(a, -1, p)!")

## 3. Малая теорема Ферма

Для простого `p` и `gcd(a, p) = 1`:

$$a^{p-1} \equiv 1 \pmod{p}$$

Следствие: $a^{-1} \equiv a^{p-2} \pmod{p}$

In [ ]:
# Проверяем малую теорему Ферма для нескольких простых чисел
primes = [5, 7, 11, 13, 17, 23, 29, 31]

for p in primes:
    all_pass = True
    for a in range(1, p):
        result = pow(a, p - 1, p)
        if result != 1:
            all_pass = False
            print(f"ОШИБКА: {a}^{p-1} = {result} (mod {p})")
    
    status = "OK" if all_pass else "FAIL"
    print(f"p = {p:2d}: a^(p-1) = 1 для всех a in Z*_p: [{status}]")

# Следствие: обратный через Ферма
print(f"\nОбратный элемент через Ферма:")
p = 17
for a in range(1, p):
    fermat_inv = pow(a, p - 2, p)
    direct_inv = pow(a, -1, p)
    assert fermat_inv == direct_inv
    print(f"{a}^(-1) = {a}^{p-2} mod {p} = {fermat_inv}")

## Упражнения

### Упражнение 1: Таблица обратных элементов

Создайте таблицу обратных элементов для GF(23). Для каждого `a` от 1 до 22 найдите `a^(-1)` и проверьте, что `a * a^(-1) = 1 (mod 23)`.

In [ ]:
# Ваш код здесь


### Упражнение 2: Китайская теорема об остатках

Реализуйте решение системы конгруэнций:
```
x ≡ 2 (mod 3)
x ≡ 3 (mod 5)
x ≡ 2 (mod 7)
```

Найдите наименьшее положительное x, используя CRT.

In [ ]:
# Ваш код здесь


### Упражнение 3: Тест простоты Ферма

Реализуйте вероятностный тест простоты на основе малой теоремы Ферма:
- Если `a^(n-1) != 1 (mod n)` для какого-то `a`, то `n` — составное.
- Проверьте для нескольких случайных `a`.
- Протестируйте на числах: 561 (число Кармайкла), 1009 (простое), 1000 (составное).

In [ ]:
# Ваш код здесь


## Челлендж: модулярное возведение в степень без pow()

Реализуйте функцию `my_modpow(base, exp, mod)`, которая вычисляет `base^exp mod mod` **без использования третьего аргумента `pow()`**. Используйте алгоритм square-and-multiply.

Требования:
1. Временная сложность O(log exp)
2. Корректность для `exp = 0`, `exp = 1`, больших значений
3. Сравните результат с `pow(base, exp, mod)` для 1000 случайных входов

In [ ]:
import random

def my_modpow(base: int, exp: int, mod: int) -> int:
    """Быстрое модулярное возведение в степень (square-and-multiply).
    
    НЕ используйте pow(base, exp, mod) — реализуйте алгоритм вручную.
    """
    # Ваш код здесь
    pass

# Тесты
assert my_modpow(3, 13, 17) == pow(3, 13, 17)
assert my_modpow(2, 0, 5) == 1
assert my_modpow(7, 1, 13) == 7

# Случайные тесты
for _ in range(1000):
    b = random.randint(2, 1000)
    e = random.randint(0, 1000)
    m = random.randint(2, 1000)
    assert my_modpow(b, e, m) == pow(b, e, m), f"Ошибка: {b}^{e} mod {m}"

print("Все 1000 тестов пройдены!")